In [1]:
import pandas as pd
import numpy as np
import tensorflow
from tensorflow import keras

2024-09-25 00:36:22.699878: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 00:36:22.711232: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 00:36:22.714636: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 00:36:22.724458: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-25 00:36:23.285413: W tensorflow/compiler/tf2

In [2]:
df = pd.read_csv("./Churn_Modelling.csv")
df.shape

(10000, 14)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [4]:
df.describe()
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [5]:
df = df.iloc[:,3:]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [7]:
Y = df.iloc[:,-1]
Y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [8]:
x = df.iloc[:,0:-1]

In [9]:
x

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [10]:
x = pd.get_dummies(x,columns=['Geography','Gender'],drop_first = True)
x

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1


In [11]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False, drop= None)
newf = df
newf['Geography']= ohe.fit_transform(df[['Geography']])
newf['Geography'].value_counts()

/home/anuj/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


1.0    5014
0.0    4986
Name: Geography, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, Y,test_size = 0.2, random_state = 1)

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [15]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [16]:
model = Sequential()

In [17]:
model.add(Dense(5, activation="relu",input_dim = 11))
model.add(Dense(3, activation="relu"))

/home/anuj/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1727204784.100421   76012 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-25 00:36:24.131994: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
S

In [18]:
model.add(Dense(1,activation='relu'))

In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 5)              │            60 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            18 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 82 (328.00 B)

 Trainable params: 82 (328.00 B)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.compile(loss='binary_crossentropy', optimizer = 'adam')

In [21]:
model.fit(x_train_scaled, y_train, epochs=300,validation_split=0.2)

Epoch 1/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 732us/step - loss: 2.0659 - val_loss: 1.3130
Epoch 2/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 448us/step - loss: 1.2006 - val_loss: 1.1086
Epoch 3/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 445us/step - loss: 1.0334 - val_loss: 0.8512
Epoch 4/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 441us/step - loss: 0.8457 - val_loss: 0.7578
Epoch 5/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step - loss: 0.7179 - val_loss: 0.5938
Epoch 6/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 474us/step - loss: 0.6463 - val_loss: 0.5684
Epoch 7/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 463us/step - loss: 0.6326 - val_loss: 0.5436
Epoch 8/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step - loss: 0.6517 - val_loss: 0.5149
Epoch 9/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 438us/step - loss: 0.5773 - val_loss: 0.5089
Epoch 10/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 479us/step - loss: 0.5987 - val_loss: 0.5061
Epoch 11/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step - loss: 0.5850 - val_loss: 0.5036
Epoch 12/300
200/20

200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 436us/step - loss: 0.4350 - val_loss: 0.4458
Epoch 68/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 462us/step - loss: 0.4388 - val_loss: 0.4446
Epoch 69/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 441us/step - loss: 0.4388 - val_loss: 0.4441
Epoch 70/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 436us/step - loss: 0.4340 - val_loss: 0.4426
Epoch 71/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 443us/step - loss: 0.4405 - val_loss: 0.4425
Epoch 72/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 450us/step - loss: 0.4227 - val_loss: 0.4419
Epoch 73/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 433us/step - loss: 0.4494 - val_loss: 0.4431
Epoch 74/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 430us/step - loss: 0.4411 - val_loss: 0.4421
Epoch 75/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 439us/step - loss: 0.4334 - val_loss: 0.4406
Epoch 76/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 463us/step - loss: 0.4369 - val_loss: 0.4397
Epoch 77/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 445us/step - loss: 0.4375 - val_loss: 0.4404
Epoch 78/300
200/200 ━━

Epoch 133/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 453us/step - loss: 0.4023 - val_loss: 0.4228
Epoch 134/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 457us/step - loss: 0.4211 - val_loss: 0.4248
Epoch 135/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 443us/step - loss: 0.3944 - val_loss: 0.4233
Epoch 136/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 437us/step - loss: 0.4043 - val_loss: 0.4242
Epoch 137/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 440us/step - loss: 0.4000 - val_loss: 0.4231
Epoch 138/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 440us/step - loss: 0.4111 - val_loss: 0.4305
Epoch 139/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 438us/step - loss: 0.4132 - val_loss: 0.4298
Epoch 140/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 432us/step - loss: 0.4137 - val_loss: 0.4304
Epoch 141/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 431us/step - loss: 0.4092 - val_loss: 0.4305
Epoch 142/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 445us/step - loss: 0.4145 - val_loss: 0.4301
Epoch 143/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 452us/step - loss: 0.4234 - val_loss: 0.4309

200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 463us/step - loss: 0.4007 - val_loss: 0.4286
Epoch 199/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step - loss: 0.3998 - val_loss: 0.4296
Epoch 200/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 476us/step - loss: 0.4081 - val_loss: 0.4225
Epoch 201/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 467us/step - loss: 0.3975 - val_loss: 0.4472
Epoch 202/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step - loss: 0.4121 - val_loss: 0.4402
Epoch 203/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 451us/step - loss: 0.3986 - val_loss: 0.4383
Epoch 204/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step - loss: 0.3818 - val_loss: 0.4233
Epoch 205/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step - loss: 0.4002 - val_loss: 0.4460
Epoch 206/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step - loss: 0.3844 - val_loss: 0.4380
Epoch 207/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 452us/step - loss: 0.3944 - val_loss: 0.4213
Epoch 208/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 463us/step - loss: 0.4016 - val_loss: 0.4246
Epoch 209/300

200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 465us/step - loss: 0.3827 - val_loss: 0.4312
Epoch 264/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 456us/step - loss: 0.4030 - val_loss: 0.4380
Epoch 265/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 459us/step - loss: 0.4116 - val_loss: 0.4390
Epoch 266/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 458us/step - loss: 0.4043 - val_loss: 0.4391
Epoch 267/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step - loss: 0.4076 - val_loss: 0.4479
Epoch 268/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 465us/step - loss: 0.4092 - val_loss: 0.4390
Epoch 269/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 463us/step - loss: 0.3951 - val_loss: 0.4471
Epoch 270/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step - loss: 0.3927 - val_loss: 0.4330
Epoch 271/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 462us/step - loss: 0.3996 - val_loss: 0.4474
Epoch 272/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step - loss: 0.4094 - val_loss: 0.4200
Epoch 273/300
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 459us/step - loss: 0.4067 - val_loss: 0.4396
Epoch 274/300

In [22]:
model.layers[0].get_weights()

[array([[-0.07836884, -0.07064916, -0.00464079,  0.20990545,  0.4636564 ],
        [-0.93553466, -0.06847983, -0.9239081 ,  0.13696286, -0.3547329 ],
        [-0.2580613 , -0.18760468, -0.08145913,  0.06457159,  0.39575467],
        [-0.18084747,  0.00131583,  0.10037434,  0.3804664 ,  0.24635725],
        [-0.02551839, -0.20070536, -0.19033638, -0.20108795, -0.32068038],
        [-0.12595484, -0.10537905, -0.05051017, -0.27967656, -0.03185732],
        [-0.6190927 , -0.90226007, -0.5682229 ,  0.79105836,  0.7922118 ],
        [ 0.02014655,  0.02196592, -0.03985848,  0.43798006,  0.42121586],
        [-0.10343501,  0.30234644,  0.18242925,  0.25529218, -0.10063097],
        [-0.1838714 , -0.00877058,  0.23321138,  0.03627417,  0.3719796 ],
        [-0.02981971, -0.2306169 ,  0.03392537, -0.5016953 , -0.4693095 ]],
       dtype=float32),
 array([ 0.8389155 ,  0.79644066,  1.0382868 , -0.02336495, -0.3067583 ],
       dtype=float32)]

In [23]:
y_log=model.predict(x_test_scaled)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 610us/step


In [24]:
y_pred = np.where(y_log>0.5,1,0)

In [25]:
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [26]:
from sklearn.metrics import accuracy_score

In [27]:
accuracy_score(y_pred,y_test)

0.834